In [23]:
import numpy as np
import pandas as pd
import pyarrow
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows',500)

In [24]:
#Load the data
rated_bookings_file = './Datasets/rated_bookings.csv'
rb=pd.read_csv(rated_bookings_file)

#Load the bookings
bookings_file = './Datasets/all_bookings2.0.csv'
book=pd.read_csv(bookings_file)


In [25]:
#Convert to Datetimes
rb['pickup_date'] = pd.to_datetime(rb['pickup_date'])
rb['last_update_date'] = pd.to_datetime(rb['last_update_date'])
rb['booking_date'] = pd.to_datetime(rb['booking_date'])
rb['rating_date'] = pd.to_datetime(rb['rating_date'])

book['pickup_date'] = pd.to_datetime(book['pickup_date'])
book['last_update_date'] = pd.to_datetime(book['last_update_date'])
book['booking_date'] = pd.to_datetime(book['booking_date'])


In [26]:
# More simple Vehicle by Category

def func_case_vehicle(v):
    if v in ['Limo 8', 'SUV Limo', 'Limo 7']:
        return('Limo')
    if v in ['Sprinter 12', 'Van', 'Minibús de 20 plazas', 'Furgoneta de 14 plazas', 'Furgoneta de 10 plazas','Monovolumen de 7 plazas']:
        return('Van')
    if v in ['Sedán']:
        return('Sedan')
    else:
        return v

#Applied to Bookings that have ratings:
rb['vehicle_category']=rb['vehicle_type'].apply(func_case_vehicle)
#rb[['vehicle_category','vehicle_type']]
#Applied to all Bookings
book['vehicle_category']=book['vehicle_type'].apply(func_case_vehicle)
#book[['vehicle_category','vehicle_type']]
filter=book['vehicle_category']=='Tour Limo'
book=book[~filter]



In [27]:
#Redefine a column for Inbound/Outbound
'''
Look at the values in route_type

rb['route_type'].value_counts()
route_type
Aeropuerto - Ciudad        10683
Ciudad - Aeropuerto         9360
Aeropuerto - Aeropuerto       79 <==== IGNORE !!
Otros                          1 <==== IGNORE !!
Name: count, dtype: int64
'''
def func_inbound_or_outbound(route_type):
    if route_type=='Aeropuerto - Ciudad':
        return 'Inbound'
    elif route_type=='Ciudad - Aeropuerto':
        return 'Outbound'
    else:
        return np.nan
#Applied to Bookings that have ratings:    
rb['in_or_outbound']=rb['route_type'].apply(func_inbound_or_outbound)
#print(rb['in_or_outbound'].value_counts())
#print(book['in_or_outbound'].unique())

#Applied to all Bookings
book['in_or_outbound']=book['route_type'].apply(func_inbound_or_outbound)
#print(book['in_or_outbound'].value_counts())
#print(book['in_or_outbound'].unique())

In [28]:
#Simple column for Airport
def set_simple_airport(route):
    if route.find('JFK')!=-1:
        return 'JFK'
    if route.find('Newark')!=-1:
        return 'EWR'
    if route.find('LaGuardia')!=-1:
        return 'LGA'
    else:
        return np.nan
#Applied to Bookings that have ratings:       
rb['airport']=rb['route'].apply(set_simple_airport)
#print(rb['airport'].value_counts())
#print(rb['airport'].unique())

#Applied to all Bookings
book['airport']=book['route'].apply(set_simple_airport)
#print(book['airport'].value_counts())
#print(book['airport'].unique())

#Get rid of these 14. Special RARE obs Airpor - to - Airport
#book[book['airport'].isna()].shape #(14, 24)
book=book[~(book['airport'].isna())]

In [29]:
#Simple column for Destination Neighborhood
def set_simple_neighborhood(route):
    if route.find('Manhattan')!=-1:
        return 'Manhattan'
    if route.find('Brooklyn')!=-1:
        return 'Brooklyn'
    if route.find('New Jersey')!=-1:
        return 'New Jersey'
    if route.find('Queens')!=-1:
        return 'Queens'
    if route.find('Bronx')!=-1:
        return 'Bronx'
    else:
        return np.nan
#Applied to Bookings that have ratings:       
rb['neighborhood']=rb['route'].apply(set_simple_neighborhood)
print(rb['neighborhood'].value_counts())
print(rb['neighborhood'].unique())

#Get rid of these Special RARE obs Airpor - to - Airport and some bad quality data from early years
print(rb[rb['in_or_outbound']=='Outbound']['route'].unique())
rb[(rb['route'].isin(['Aeropuerto JFK - Manhattan','Aeropuerto Newark - Manhattan'])) & (rb['in_or_outbound']=='Outbound')] #<== Good now!
rb=rb[~(rb['neighborhood'].isna())]



#Applied to all Bookings
book['neighborhood']=book['route'].apply(set_simple_neighborhood)
print(book['neighborhood'].value_counts())
print(book['neighborhood'].unique())
print(book[book['neighborhood'].isna()].shape) #(159, 25)
print(book[book['neighborhood'].isna()])

#Get rid of these Special RARE obs Airpor - to - Airport and some bad quality data from early years
book=book[~(book['neighborhood'].isna())]
book=book[~((book['route'].isin(['Aeropuerto JFK - Manhattan','Aeropuerto Newark - Manhattan'])) & (book['in_or_outbound']=='Outbound'))]


neighborhood
Manhattan     18950
Brooklyn        458
New Jersey      333
Queens          299
Bronx            37
Name: count, dtype: int64
['Manhattan' 'Queens' 'Brooklyn' 'New Jersey' nan 'Bronx']
['Manhattan - Aeropuerto JFK' 'Manhattan - Aeropuerto Newark'
 'Manhattan - Aeropuerto LaGuardia' 'Queens - Aeropuerto JFK'
 'Brooklyn - Aeropuerto JFK' 'Brooklyn - Aeropuerto Newark'
 'New Jersey - Aeropuerto JFK' 'Queens - Aeropuerto Newark'
 'New Jersey - Aeropuerto LaGuardia' 'Brooklyn - Aeropuerto LaGuardia'
 'New Jersey - Aeropuerto Newark' 'Bronx - Aeropuerto JFK'
 'Queens - Aeropuerto LaGuardia' 'Bronx - Aeropuerto LaGuardia']
neighborhood
Manhattan     63811
Brooklyn       1480
New Jersey     1072
Queens          928
Bronx           121
Name: count, dtype: int64
['Manhattan' 'Brooklyn' 'New Jersey' 'Queens' nan 'Bronx']
(159, 25)
           status   supplier booking_id roundtrip_id              name  \
2526          NaN       DH2L    T106630          NaN      Miren Jasone   
2530   

In [30]:
#Take a look at 'supplier'
#rb['supplier'].unique()
#Let's rename them
def rename_supplier(name):
    if name=='NewDay':
        return 'LES Radio Dispatch'
    if name=='HUGO':
        return 'El Master'
    if name=='Best Limo':
        return 'Trusty Limo'
    if name=='MarcoBPS':
        return 'Marconni LLC'
    if name=='Patricio':
        return 'Brother_2'
    if name=='800S':
        return 'Super Vans'
    if name=='LXNY':
        return 'BowTie Corp'
    if name=='DH2L':
        return 'Vargas Trips'
    else:
        return name

#Applied to Bookings that have ratings:           
rb['supplier']=rb['supplier'].apply(rename_supplier)
print(rb['supplier'].value_counts(dropna=False))
print(rb[rb['supplier'].isna()].shape) #(9,31)
rb=rb[~(rb['supplier'].isna())]


#Applied to all Bookings
book['supplier']=book['supplier'].apply(rename_supplier)
#It is normal to have some reservations with no supplier. They were cancelled before having one.
#Also, we have some bookings that are for the future and are not yet assigned to a supplier.
print(book['supplier'].value_counts(dropna=False))

#There are 83 reservations that don't have supplier and were not cancelled before they were asigned.
book=book[~((book['supplier'].isna()) & (book['pickup_date'].dt.year<2023)& (book['status']!='cancelado'))]

supplier
Marconni LLC          4006
BowTie Corp           3890
Trusty Limo           3869
Brother_2             3066
El Master             2153
Vargas Trips          2011
Super Vans             964
LES Radio Dispatch     104
NaN                      9
Golden Class Limo        3
Nestor                   1
Johnny Limo              1
Name: count, dtype: int64
(9, 31)
supplier
Marconni LLC          13509
BowTie Corp           11634
Trusty Limo           10055
Brother_2              9927
Vargas Trips           6303
El Master              5682
NaN                    5308
Super Vans             3798
LES Radio Dispatch     1098
FCLimo                   20
Golden Class Limo        19
PeterLimo                17
Nestor                   17
Efrain                    3
Camila                    2
Johnny Limo               2
FRANKLYN                  1
Name: count, dtype: int64


In [31]:
#Time from Booking to Pickup
#Calculate number of days.
#Express it in groups that make sense.

#rb[['pickup_date','booking_date']]
def time_delta_only_in_days(td):
    if td.days+((td.seconds/3600)/24) <= 0.29:
        return np.nan
    else:
        return td.days+((td.seconds/3600)/24) #convert seconds to hours and hours to days.

def make_days_in_advance_groups(n):
    if (n<=0.6):
        return 'G01_12 hour'
    elif (n>0.6) & (n<=1.2):
        return 'G02_24 hour'
    elif (n>1.2) & (n<=2.4):
        return 'G03_48 hour'
    elif (n>2.4) & (n<=4.8):
        return 'G04_4 day'
    elif (n>4.8) & (n<=8):
        return 'G05_1 week'
    elif (n>8) & (n<=16):
        return 'G06_2 week'
    elif (n>16) & (n<=34):
        return 'G07_1 month'
    elif (n>34) & (n<=66):
        return 'G08_2 month'
    elif (n>66) & (n<=100):
        return 'G09_3 month'
    elif (n>100) & (n<=190):
        return 'G10_6 month'
    elif (n>190):
        return 'G11_> 6 month'
    else:
        return np.nan

#Applied to Bookings that have ratings: 
rb['days_in_advance']=pd.DataFrame(rb['pickup_date']-rb['booking_date'])
rb['days_in_advance']=rb['days_in_advance'].apply(time_delta_only_in_days)
print(rb['days_in_advance'].describe())

rb['day_in_advance_groups']=rb['days_in_advance'].apply(make_days_in_advance_groups)
rb['day_in_advance_groups'].value_counts(dropna=False)

#rb[rb['days_in_advance']<.4].sort_values(by='days_in_advance')


#Applied to all Bookings
book['days_in_advance']=pd.DataFrame(book['pickup_date']-book['booking_date'])
book['days_in_advance']=book['days_in_advance'].apply(time_delta_only_in_days)
print(book['days_in_advance'].describe())

book['day_in_advance_groups']=book['days_in_advance'].apply(make_days_in_advance_groups)
book['day_in_advance_groups'].value_counts(dropna=False)

#book[book['days_in_advance']<.25].sort_values(by='days_in_advance')


count    19999.000000
mean        30.504223
std         39.563313
min          0.292361
25%          6.972222
50%         16.423611
75%         36.985069
max        340.183333
Name: days_in_advance, dtype: float64
count    67050.000000
mean        39.091079
std         53.525333
min          0.292361
25%          7.416319
50%         18.411458
75%         46.065625
max        769.075694
Name: days_in_advance, dtype: float64


day_in_advance_groups
G07_1 month      14878
G06_2 week       12882
G08_2 month       9890
G05_1 week        5744
G10_6 month       4952
G04_4 day         4682
G09_3 month       4552
G03_48 hour       4127
G02_24 hour       2690
G11_> 6 month     2254
G01_12 hour        399
NaN                262
Name: count, dtype: int64

In [32]:
#Departure Country
country_dict={'madrid':'Spain'
,'barcelona':'Spain'
,'paris':'France'
,'buenos aires':'Argentina'
,'lisboa':'Portugal'
,'miami':'USA'
,'londres':'England'
,'santiago de chile':'Chile'
,'zurich':'Switzerland'
,'malaga':'Spain'
,'santiago':'Chile'
,'amsterdam':'Netherlands'
,'bogota':'Colombia'
,'orlando':'USA'
,'mad':'Spain'
,'mexico':'Mexico'
,'ciudad de mexico':'Mexico'
,'nueva york':'Spain'
,'dublin':'Ireland'
,'frankfurt':'Germany'
,'málaga':'Spain'
,'españa':'Spain'
,'valencia':'Spain'
,'bcn':'Spain'
,'argentina':'Argentina'
,'panama':'Panama'
,'london':'England'
,'montevideo':'Uruguay'
,'roma':'Italy'
,'ciudad de méxico':'Mexico'
,'las vegas':'USA'
,'munich':'Germany'
,'milano':'Italy'
,'guadalajara':'Mexico'
,'puerto rico':'Puerto Rico'
,'lima':'Peru'
,'sevilla':'Spain'
,'toronto':'Canada'
,'palma de mallorca':'Spain'
,'atlanta':'USA'
,'genève':'Switzerland'
,'chicago':'USA'
,'san francisco':'USA'
,'bruxelles':'Belgium'
,'paris cdg':'France'
,'madrid (spain)':'Spain'
,'geneve':'Switzerland'
,'bruselas':'Belgium'
,'los angeles':'USA'
,'bilbao':'Spain'
,'madrid (españa)':'Spain'
,'mexico city':'Mexico'
,'washington':'USA'
,'dallas':'USA'
,'boston':'USA'
,'san juan, puerto rico':'Puerto Rico'
,'houston':'USA'
,'oporto':'Portugal'
,'panamá':'Panama'
,'san salvador':'El Salvador'
,'cancun':'Mexico'
,'sao paulo':'Brazil'
,'lisbon':'Portugal'
,'monterrey':'Mexico'
,'san juan':'Puerto Rico'
,'tenerife sur':'Spain'
,'nice':'France'
,'montreal':'Canada'
,'tenerife':'Spain'
,'aeropuerto jfk':'Spain'
,'francfort':'Germany'
,'san pablo':'Brazil'
,'casablanca':'Marocco'
,'parís':'France'
,'buenos aires, argentina':'Argentina'
,'brussels':'Belgium'
,'santiago, chile':'Chile'
,'bogotá':'Colombia'
,'porto':'Portugal'
,'barcelone':'Spain'
,'paris orly':'France'
,'santo domingo':'Dominican Rep.'
,'são paulo':'Brazil'
,'barcelona (españa)':'Spain'
,'méxico':'Mexico'
,'charlotte':'USA'
,'buenos aires - argentina':'Argentina'
,'santa cruz de tenerife':'Spain'
,'alicante':'Spain'
,'madrid, spain':'Spain'
,'milano malpensa':'Italy'
,'geneva':'Switzerland'
,'seattle':'USA'
,'france':'France'
,'santiago chile':'Chile'
,'guatemala':'Guatemala'}

def translate_city_of_origin_into_country(city):
    if city in country_dict.keys():
        return country_dict[city]
    else:
        return np.nan
    
rb['city_of_departure']=rb['city_of_departure'].astype(str)
rb['city_of_departure']=rb['city_of_departure'].apply(str.lower)
rb['city_of_departure']=rb['city_of_departure'].apply(str.strip)
rb['country_departure']=rb['city_of_departure'].apply(translate_city_of_origin_into_country)
rb['country_departure'].value_counts(dropna=False)

zone_dict={'Spain':'EU'
,'France':'EU'
,'USA':'USA'
,'Argentina':'LATAM'
,'Mexico':'LATAM'
,'Chile':'LATAM'
,'England':'EU'
,'Switzerland':'EU'
,'Germany':'EU'
,'Colombia':'LATAM'
,'Netherlands':'EU'
,'Italy':'EU'
,'Puerto Rico':'LATAM'
,'Panama':'LATAM'
,'Ireland':'EU'
,'Belgium':'EU'
,'Portugal':'EU'
,'Canada':'Canada'
,'Brazil':'LATAM'
,'Uruguay':'LATAM'
,'Peru':'LATAM'
,'El Salvador':'LATAM'
,'Marocco':''
,'Dominican Rep.':'LATAM'
,'Guatemala':'LATAM'}

def translate_country_of_origin_into_zone(city):
    if city in zone_dict.keys():
        return zone_dict[city]
    else:
        return np.nan
    
rb['zone_departure']=rb['country_departure'].apply(translate_country_of_origin_into_zone)
rb['zone_departure'].value_counts(dropna=False)

zone_departure
NaN       10695
EU         7848
LATAM      1086
USA         390
Canada       37
             12
Name: count, dtype: int64

In [33]:
#Boolean field for reservations that have been updated at some point.

#Applied to Bookings that have ratings:       

rb.loc[rb['status'].isna(), ['has_been_updated'] ]= False
rb.loc[rb['status']=='modificado', ['has_been_updated'] ]= True
rb.loc[rb['status']=='cancelado', ['has_been_updated'] ]= np.nan
rb['has_been_updated'].astype('boolean')

print(rb['has_been_updated'].dtypes)
print(rb['has_been_updated'].unique())
print(rb['has_been_updated'].value_counts())
print(rb['has_been_updated'].isna().sum())

#Applied to all Bookings
book.loc[book['status'].isna(), ['has_been_updated'] ]= False
book.loc[book['status']=='modificado', ['has_been_updated'] ]= True
book.loc[book['status']=='cancelado', ['has_been_updated'] ]= np.nan
book['has_been_updated'].astype('boolean')


print(book['has_been_updated'].dtypes)
print(book['has_been_updated'].unique())
print(book['has_been_updated'].value_counts())
print(book['has_been_updated'].isna().sum())

object
[False True nan]
has_been_updated
False    15526
True      4394
Name: count, dtype: int64
148
object
[False True nan]
has_been_updated
False    49536
True     12843
Name: count, dtype: int64
4933


/var/folders/md/9_b_6n6s0jj4pbk1s4kdtxj00000gn/T/ipykernel_1694/1612362381.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  rb.loc[rb['status'].isna(), ['has_been_updated'] ]= False
/var/folders/md/9_b_6n6s0jj4pbk1s4kdtxj00000gn/T/ipykernel_1694/1612362381.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  book.loc[book['status'].isna(), ['has_been_updated'] ]= False


In [34]:
'''
#This executes the connection to Geoapify to obtain GPS coordinates of destinations
#Geoapify

import requests
import urllib
import time

def get_coordinates_from_geoapify(lookup_address, lookup_hotel):
    if (len(lookup_address)>5):
        address_url=urllib.parse.quote(lookup_address)
        url = 'https://api.geoapify.com/v1/geocode/search?text='+address_url+'&format=json&apiKey=7baacd7506b544fe9f684cc38dd8e8fb'
        url          
        response = requests.get(url)
        response_json=response.json()
        if response_json["results"][0]["rank"]["confidence"]>=.5:
                add_lon= response_json["results"][0]["lon"]
                add_lat= response_json["results"][0]["lat"]
                return (add_lat, add_lon)
        elif (len(lookup_hotel)>5):
                add_lat,add_lon=get_coordinates_from_geoapify(lookup_hotel,'')
                return (add_lat, add_lon)
    elif (len(lookup_hotel)>5):
            add_lat,add_lon=get_coordinates_from_geoapify(lookup_hotel,'')
            return (add_lat, add_lon)
    else:
          return (np.nan, np.nan)
n=0
for i in rb.index:
        print(i)
        if pd.isna(rb.loc[i]['address_lat']):
                try:
                        add_lat,add_lon=get_coordinates_from_geoapify(rb.loc[i]['address'] ,rb.loc[i]['hotel'])
                        rb.loc[i, ['address_lat']]=add_lat
                        rb.loc[i, ['address_loc']]=add_lon
                except: 
                        print(rb.loc[i]['address'] ,rb.loc[i]['hotel'])
                        n+=1
                        time.sleep(.25)
        else:
                n+=1
'''

'\n#This executes the connection to Geoapify to obtain GPS coordinates of destinations\n#Geoapify\n\nimport requests\nimport urllib\nimport time\n\ndef get_coordinates_from_geoapify(lookup_address, lookup_hotel):\n    if (len(lookup_address)>5):\n        address_url=urllib.parse.quote(lookup_address)\n        url = \'https://api.geoapify.com/v1/geocode/search?text=\'+address_url+\'&format=json&apiKey=7baacd7506b544fe9f684cc38dd8e8fb\'\n        url          \n        response = requests.get(url)\n        response_json=response.json()\n        if response_json["results"][0]["rank"]["confidence"]>=.5:\n                add_lon= response_json["results"][0]["lon"]\n                add_lat= response_json["results"][0]["lat"]\n                return (add_lat, add_lon)\n        elif (len(lookup_hotel)>5):\n                add_lat,add_lon=get_coordinates_from_geoapify(lookup_hotel,\'\')\n                return (add_lat, add_lon)\n    elif (len(lookup_hotel)>5):\n            add_lat,add_lon=get_c

In [35]:
#Load the Coordinates
coordinates_file = 'Datasets/location_coordinates.csv'
coordinates=pd.read_csv(coordinates_file)

merged=pd.merge(rb,coordinates, how='left',left_on='booking_id',right_on='booking_id')
merged.shape
rb=merged

In [36]:
'''
# Google API Key ***************************************
                 
! pip install -U googlemaps

import requests
import urllib.parse
import os
import geocoder

import googlemaps
from datetime import datetime

gmaps = googlemaps.Client(key='***************************************')

# Geocoding an address
geocode_result = gmaps.geocode('575 grand street 10002 NY, NY')

lat = geocode_result[0]["geometry"]["location"]["lat"]
lon = geocode_result[0]["geometry"]["location"]["lng"]

'''

'\n# Google API Key ***************************************\n                 \n! pip install -U googlemaps\n\nimport requests\nimport urllib.parse\nimport os\nimport geocoder\n\nimport googlemaps\nfrom datetime import datetime\n\ngmaps = googlemaps.Client(key=\'***************************************\')\n\n# Geocoding an address\ngeocode_result = gmaps.geocode(\'575 grand street 10002 NY, NY\')\n\nlat = geocode_result[0]["geometry"]["location"]["lat"]\nlon = geocode_result[0]["geometry"]["location"]["lng"]\n\n'

In [37]:
#After all the changes, we proceed to save the changes in a file.
#Specially now that getting the coordinates takes a long time.
import os
outname = 'rated_bookings_with_location.csv'
outdir = './Datasets'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname)    

rb.to_csv(fullname, header=True, index=False)
print("Saved!")

import os
outname = 'all_bookings3.0.csv'
outdir = './Datasets'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname)    

book.to_csv(fullname, header=True, index=False)
print("Saved!")


Saved!
Saved!
